In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
import pandas as pd
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer  # Optional: for stemming
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# Ensure NLTK stopwords are downloaded
# nltk.download('stopwords')
# nltk.download("punkt")

# Load Training Data
train_path = "/content/gdrive/MyDrive/Genre Classification Dataset/train_data.txt"
train_data = pd.read_csv(train_path, sep=':::', names=['Title', 'Genre', 'Description'], engine='python')

# Load Test Data
test_path = "/content/gdrive/MyDrive/Genre Classification Dataset/test_data.txt"
test_data = pd.read_csv(test_path, sep=':::', names=['Id', 'Title', 'Description'], engine='python')

stemmer = LancasterStemmer()  # Optional: for stemming
stop_words = set(stopwords.words('english'))


def preprocess_text(text):
  """
  This function performs all preprocessing steps on text data.
  """
  # text = text.lower()  # Lowercase all characters
  text = re.sub(r'@\S+', '', text)  # Remove Twitter handles
  text = re.sub(r'http\S+', '', text)  # Remove URLs
  text = re.sub(r'pic.\S+', '', text)
  text = re.sub(r"[^a-zA-Z+']", ' ', text)  # Keep only characters
  text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text + ' ')  # Keep words with length > 1 only
  text = "".join([i for i in text if i not in string.punctuation])
  words = nltk.word_tokenize(text)
  words = [word for word in words if word not in stop_words and len(word) > 2]  # Remove stopwords and short words
  # Optional: Apply stemming if desired
  # words = [stemmer.stem(word) for word in words]

  return " ".join(words).strip()  # Remove spaces and return preprocessed text


train_data['Tokens'] = train_data['Description'].apply(preprocess_text)
test_data['Tokens'] = test_data['Description'].apply(preprocess_text)
# print(train_data.head())


In [13]:
# TF-IDF Vectorization

tf_vector = TfidfVectorizer()

X_train = tf_vector.fit_transform(train_data["Tokens"])
X_test = tf_vector.transform(test_data["Tokens"])
dic_vocabulary = tf_vector.vocabulary_
# X_names = tf_vector.get_feature_names_out()

X = X_train
y = train_data["Genre"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

lr_y_pred = model_lr.predict(X_val)
lr_accuracy = accuracy_score(y_val, lr_y_pred)
print("Logistic Regression Accuracy on Validation Set:", lr_accuracy)


Logistic Regression Accuracy on Validation Set: 0.5823111684958038


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
